### Jul AdventKalender D22

https://adventofcode.com/2022/day/22

#### Day 22.1 

Given a file, in which the first half shows a map (open tiles '.' and solid walls '#'), and second half is a description of the path you must follow. 

The path consists of alternating numbers and letters:

* A **number** indicates the **number of tiles to move** in the direction you are facing. If you run into a wall, you stop moving forward and continue with the next instruction.
* A **letter** indicates whether to turn 90 degrees **clockwise (R)** or **counterclockwise (L)**. Turning happens in-place; it does not change your current tile.

For example:

                ...#
                .#..
                #...
                ....
        ...#.......#
        ........#...
        ..#....#....
        ..........#.
                ...#....
                .....#..
                .#......
                ......#.

        10R5L5R10L4R5L5
    
Begin the path in the leftmost open tile of the top row of tiles and facing to the right. If a movement instruction would take you off of the map, you **wrap around** to the other side of the board.

Rows start from 1 at the top and count downward; columns start from 1 at the left and count rightward. Facing is 0 for right (>), 1 for down (v), 2 for left (<), and 3 for up (^). The final password is the sum of 1000 times the **row**, 4 times the **column**, and the **facing**.

Follow the path given in the monkeys' notes. What is the final password?

In [1]:
import numpy as np
import re

def _parseMap(data_map):
    nrow = len(data_map)
    ncol = 0
    for r in data_map:
        ncol = max(ncol, len(r))
    data_map_arr = np.zeros((nrow,ncol)) # 0 is nothing
    for i in range(nrow):
        for j in range(len(data_map[i])):
            if data_map[i][j]=='.': data_map_arr[i,j] = 1    # 1 is open tile
            elif data_map[i][j]=='#': data_map_arr[i,j] = -1 # -1 is solid wall
    return data_map_arr

def _parsePath(path):
    steps = list(map(int, re.findall('[0-9]+', path)))
    directions = re.findall('[A-Z]+', path)
    return steps, directions

def readFile(file_name):
    data_map = []
    path = ''
    f = open(file_name, "r")
    is_path = False
    while True:
        line = f.readline()
        if not line:
            break
        if line.strip('\n') == '':
            is_path = True
        if is_path: path = line.strip('\n')
        else: data_map.append(line.strip('\n'))
    f.close()
    data_map_arr = _parseMap(data_map)
    path_steps, path_directions = _parsePath(path)
    return data_map_arr, path_steps, path_directions

In [2]:
def getStartPos():
    global data_map_arr
    col_id = np.where(data_map_arr[0,:] == 1)[0][0] # first/leftmost 1/opentile in the first/top row
    return (0,col_id)

In [8]:
dir_steps = [np.array([0,1]),np.array([1,0]),np.array([0,-1]),np.array([-1,0])] # 0 right, 1 down, 2 left, 3 up
  
# given a row or column and a direction, get the valid tile (not zero)
def _getWrappedIndex1(pos_to, dir_cur):
    global data_map_arr
    if (dir_cur in [0,2]): target_list = data_map_arr[pos_to[0],:]
    else: target_list = data_map_arr[:,pos_to[1]]
    if dir_cur in [2,3]:
        for i in range(len(target_list)-1,-1,-1):
            if target_list[i]!=0:
                if dir_cur == 2: pos_to[1] = i
                else: pos_to[0] = i
                break
    elif dir_cur in [0,1]:
        for i in range(len(target_list)):
            if target_list[i]!=0:
                if dir_cur == 1: pos_to[0] = i
                else: pos_to[1] = i
                break
    return pos_to, dir_cur

# turn 90 degrees based on the given direction: clockwise (R) or counterclockwise (L)
def _takeDirection(RL_dir, dir_cur):
    return (dir_cur+1)%4 if RL_dir == 'R' else (dir_cur-1)%4
    
# move n steps from current position in current direction
def _takeMove(n_step, pos_cur, dir_cur, wrappedFunc):
    global data_map_arr
    for _ in range(n_step):
        pos_to = pos_cur + dir_steps[dir_cur]
        dir_to = dir_cur
        # get valid pos_to (wrap if nessesary)
        # left right => col index, pos_to[1]
        if (dir_cur in [0,2]) and (pos_to[1]<0 or pos_to[1]>=data_map_arr.shape[1] or data_map_arr[tuple(pos_to)]==0):
            pos_to,dir_to = wrappedFunc(pos_to, dir_cur)
        # up down => row index, pos_to[0]
        if dir_cur in [1,3] and (pos_to[0]<0 or pos_to[0]>=data_map_arr.shape[0] or data_map_arr[tuple(pos_to)]==0):
            pos_to,dir_to = wrappedFunc(pos_to, dir_cur)  
        pos_to = tuple(pos_to)
        if data_map_arr[pos_to] == 1: # opentile can move, otherwise wall stay put
            pos_cur = pos_to
            dir_cur = dir_to
    return pos_cur,dir_cur

def walkPath(pos_start, dir_start, wrappedFunc = _getWrappedIndex1):
    global data_map_arr, path_steps, path_directions
    pos_cur, dir_cur = pos_start, dir_start
    for i in range(max(len(path_steps),len(path_directions))):
        if i < len(path_steps):
            pos_cur,dir_cur = _takeMove(path_steps[i], pos_cur, dir_cur, wrappedFunc) # move steps
        if i < len(path_directions):
            dir_cur = _takeDirection(path_directions[i], dir_cur) # turn direction
    return pos_cur, dir_cur

def getPwd(pos_cur, dir_cur):
    return 1000 * (pos_cur[0]+1) + 4 * (pos_cur[1]+1) + dir_cur

In [9]:
data_map_arr, path_steps, path_directions = readFile('data/input22.txt')
pos_start, dir_start = getStartPos(), 0 # 0 right, 1 down, 2 left, 3 up
pos_cur, dir_cur = walkPath(pos_start, dir_start)

In [10]:
getPwd(pos_cur, dir_cur)

117054

#### Day 21.2

The wrapping rules are different now. The map is actually a cube!

In the example above, the six (smaller, 4x4) faces of the cube are:

                1111
                1111
                1111
                1111
        222233334444
        22223333444A*
        222233334444
        2D2233334444  *
         *      555566B6
                55556666
                55556666
                55C56666
                  *

If you are at A and move to the right, you would arrive at B facing down; if you are at C and move down, you would arrive at D facing up.

What is the final password now?

In [11]:
# For now it only works on the following pattern:
#                 11112222
#                 11112222
#                 11112222
#                 11112222
#                 3333
#                 3333
#                 3333
#                 3333 
#             44445555
#             44445555
#             44445555
#             44445555
#             6666
#             6666
#             6666
#             6666
# The following function has not been organized yet
def _getWrappedIndex2(pos_to, dir_cur):
    global data_map_arr, width
    dir_to = dir_cur
    #print(pos_to)
    if dir_cur in [0,2]: # right left
        if pos_to[0]//width == 0: 
            pos_to[0] = (width-1-(pos_to[0]%width) + (pos_to[0]//width+2)*width) # row id
            dir_to = 2 if dir_cur==0 else 0
        elif pos_to[0]//width == 1: 
            if dir_cur==2: # left
                pos_to[1] = (pos_to[0]%width + ((pos_to[1]+1)//width-1)*width) # col id
                dir_to = 1 # down
            else: # right
                pos_to[1] = (pos_to[0]%width + ((pos_to[1]-1)//width+1)*width) # col id
                dir_to = 3 # up
        elif pos_to[0]//width == 2: 
            pos_to[0] = (width-1-(pos_to[0]%width) + (pos_to[0]//width-2)*width) # row id
            dir_to = 0 if dir_cur==2 else 2
        elif pos_to[0]//width == 3: 
            if dir_cur==2: # left
                pos_to[1] = (pos_to[0]%width + ((pos_to[1]+1)//width+1)*width) # col id
                dir_to = 1 # down
            else: # right
                pos_to[1] = (pos_to[0]%width + ((pos_to[1]-1)//width+1)*width) # col id
                dir_to = 3 # up
    elif dir_cur in [1,3]:
        if pos_to[1]//width==0: 
            if dir_cur==3: # up
                pos_to[0] = (pos_to[1]%width + ((pos_to[0]+1)//width-1)*width) # row id
                dir_to = 0
            else:
                pos_to[1] = (pos_to[1]%width + (pos_to[1]//width+2)*width) # col id
                dir_to = 1
        elif pos_to[1]//width==1: 
            if dir_cur==3: # up
                pos_to[0] = (pos_to[1]%width + ((pos_to[0]+1)//width+3)*width) # row id
                dir_to = 0
            else:
                pos_to[0] = (pos_to[1]%width + ((pos_to[0]-1)//width+1)*width) # row id
                dir_to = 2
        elif pos_to[1]//width==2: 
            if dir_cur==3: # up
                pos_to[1] = (pos_to[1]%width + (pos_to[1]//width-2)*width) # col id
            else:
                pos_to[0] = (pos_to[1]%width + ((pos_to[0]-1)//width+1)*width) # row id
                dir_to = 2
    #print(pos_to)
    return  _getWrappedIndex1(pos_to, dir_to)

In [12]:
data_map_arr, path_steps, path_directions = readFile('data/input22.txt')
pos_start, dir_start = getStartPos(), 0 # 0 right, 1 down, 2 left, 3 up
width = 50
pos_cur, dir_cur = walkPath(pos_start, dir_start, _getWrappedIndex2)

In [13]:
getPwd(pos_cur, dir_cur)

162096